In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('churn_analysis.csv')

In [3]:
data.drop(data.columns[0], axis=1, inplace=True)

# 1

> Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Какой критерий подходит для решения этой задачи? Сколько достигаемых уровней значимости оказались меньше, чем α=0.05?

In [4]:
control_group = data[data['treatment'] == 1]

In [5]:
# массив таблиц сопряженности
states_contingency_tables = []
# значения, которые принимают state и churn на контрольной группе
control_states = control_group['state'].unique()
control_churns = control_group['churn'].unique()
for i in range(len(control_states) - 1):
    for j in range(i + 1, len(control_states)):
        contingency_data = {}
        for churn in control_churns:
            contingency_data.update({churn: [len(control_group[(control_group['state'] ==
                                                                control_states[i]) &
                                                (control_group['churn'] == churn)]),
                                             len(control_group[(control_group['state'] ==
                                                                control_states[j]) &
                                                (control_group['churn'] == churn)])]})
        # формируем таблицу сопряженности
        contingency_table = pd.DataFrame(contingency_data, index=[control_states[i], control_states[j]])
        states_contingency_tables.append(contingency_table)

In [6]:
control_states

array(['KS', 'OH', 'LA', 'IA', 'VT', 'CO', 'WY', 'MO', 'GA', 'AK', 'MA',
       'NJ', 'AR', 'MI', 'IN', 'MN', 'SD', 'TX', 'DC', 'NY', 'OR', 'AZ',
       'MS', 'UT', 'ME', 'ID', 'FL', 'NV', 'WA', 'WV', 'NH', 'VA', 'NM',
       'KY', 'MT', 'PA', 'RI', 'DE', 'OK', 'NE', 'CT', 'WI', 'CA', 'NC',
       'MD', 'TN', 'AL', 'IL', 'ND', 'HI', 'SC'], dtype=object)

In [7]:
states_contingency_tables[0]

,False.,True.
KS,18,7
OH,22,2


In [8]:
states_contingency_tables[-1]

,False.,True.
HI,12,1
SC,16,5


In [9]:
len(states_contingency_tables)

1275

Для решения этой задачи подходит точный критерий Фишера.

In [10]:
from scipy.stats import fisher_exact
alpha = 0.05
# счетчик, отвечающий за количество встретившихся достигаемых уровней значимости, меньших alpha
cnt = 0
for contingency_table in states_contingency_tables:
    oddsratio, p_value = fisher_exact(contingency_table)
    if p_value < alpha:
        cnt += 1
cnt

10

# 2

> Посчитайте корреляции Пирсона и Спирмена между day_calls и mes_estim на всех данных, оцените их значимость, дайте интерпретацию результата.

In [11]:
pearson_corr = data['day_calls'].corr(data['mes_estim'], method='pearson')
pearson_corr

-0.051794350587572618

In [12]:
spearman_corr = data['day_calls'].corr(data['mes_estim'], method='spearman')
spearman_corr

0.043349880533927444

In [13]:
n = len(data)
n

3333

Воспользуемся критерием Стьюдента. Вычислим статистику $T = \frac{r\sqrt{n - 2}}{\sqrt{1 - r^2}}$ - эта статистика подчиняется распределению Стьюдента с $n - 2$ степенями свободы, где $n = 3333$ - размер выборки, $r$ - посчитанный коэффициент корреляции. Далее сравним модуль полученного результата с табличным критическим значением распределения Стьюдента при уровне значимости $\alpha = 0.05$ и $n - 2$ степенями свободы - оно примерно равно $1.96$.

In [14]:
T_pearson = pearson_corr * np.sqrt(n - 2) / np.sqrt(1 - pearson_corr ** 2)
T_pearson

-2.9933191375739687

In [15]:
T_spearman = spearman_corr * np.sqrt(n - 2) / np.sqrt(1 - spearman_corr ** 2)
T_spearman

2.5042845313829765

Видно, что в каждом из случаев модуль полученного значения статистики больше табличного критического значения. Это означает, что нулевая гипотеза об отсутствии зависимости между признаками отвергается, то есть данные позволяют считать, что связь между признаками существенная. И это можно интерпретировать: наверное, если человек мало пользуется интернет мессенджерами (значение mes_estim мало), то он много звонит по телефону (значение day_calls велико), и наоборот.

# 3

> Посчитайте значение коэффицента корреляции Крамера между штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1). Проверьте гипотезу об отсутствии связи между этими признаками.

Считаем хи-квадрат:

In [16]:
chi2 = 0
for state in control_group['state'].unique():
    for churn in control_group['churn'].unique():
        observed_freq = len(control_group[(control_group['state'] == state) &
                                          (control_group['churn'] == churn)])
        expected_freq = (len(control_group[control_group['state'] == state]) *
                         len(control_group[control_group['churn'] == churn]) /
                         len(control_group))
        chi2 += (observed_freq - expected_freq) ** 2 / expected_freq
print(chi2)

44.0527123669255


Этот же результат можно получить, использовав встроенную функцию chi2_contingency в библиотеке scipy.stats, предварительно составив таблицу сопряженности для столбцов state и churn.

In [17]:
observed = []
for state in control_group['state'].unique():
    observed_state = []
    for churn in control_group['churn'].unique():
        observed_freq = len(control_group[(control_group['state'] == state) &
                                          (control_group['churn'] == churn)])
        observed_state.append(observed_freq)
    observed.append(observed_state)
print(observed)

[[18, 7], [22, 2], [15, 0], [13, 2], [18, 5], [17, 5], [20, 2], [17, 4], [19, 2], [19, 1], [17, 7], [20, 4], [11, 5], [17, 5], [20, 3], [19, 2], [20, 3], [14, 6], [18, 1], [27, 6], [18, 3], [17, 2], [20, 4], [18, 4], [20, 8], [16, 2], [18, 2], [18, 6], [16, 7], [27, 5], [10, 2], [20, 3], [27, 2], [21, 4], [22, 4], [14, 4], [18, 4], [16, 1], [11, 2], [21, 3], [25, 5], [22, 2], [10, 5], [16, 3], [19, 5], [11, 1], [25, 5], [11, 1], [21, 3], [12, 1], [16, 5]]


In [18]:
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(observed)
chi2

44.052712366925476

Искомый коэффициент корреляции Крамера:

In [19]:
cramer_v = np.sqrt(chi2 / len(control_group))
cramer_v

0.20039321502033317

Воспользуемся критерием хи-квадрат. В данном случае статистикой является уже вычисленная величина chi2, которая подчиняется хи-квадрат распределению с количеством степеней свободы $50$. Табличное критическое значение соответствующего хи-квадрат распределения при уровне значимости $\alpha = 0.05$ примерно равно $67.5$, поэтому гипотеза об отсутствии зависимости не отвергается, и связь не существенная.

# 4

> Проведите анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1). Что можно сказать об этих двух методах (treatment = 0, treatment = 2)? Одинаковы ли они с точки зрения эффективности? Каким бы методом вы бы посоветовали воспользоваться компании?

In [20]:
control_group['churn'].value_counts()

False.    917
True.     180
Name: churn, dtype: int64

In [21]:
len(control_group['churn'])

1097

In [22]:
treatment0 = data[data['treatment'] == 0]
treatment0['churn'].value_counts()

False.    968
True.     165
Name: churn, dtype: int64

In [23]:
len(treatment0['churn'])

1133

In [24]:
treatment2 = data[data['treatment'] == 2]
treatment2['churn'].value_counts()

False.    965
True.     138
Name: churn, dtype: int64

In [25]:
len(treatment2['churn'])

1103

Воспользуемся критерием для долей. Для каждой пары выборок посчитаем статистики, равные

$$
T_{ij} = \frac{\frac{m_i}{n_i} - \frac{m_j}{n_j}}{\sqrt{\frac{m_i + m_j}{n_i + n_j} \left(1 - \frac{m_i + m_j}{n_i + n_j}\right) \left(\frac{1}{n_i} + \frac{1}{n_j}\right)}},
$$

где $m_i$ и $m_j$ - число оставшихся пользователей в $i$-ой и $j$-ой выборках, $n_i$ и $n_j$ - размеры соответствующих выборок.

In [26]:
m0 = len(treatment0[treatment0['churn'] == 'False.'])
n0 = len(treatment0)
m1 = len(control_group[control_group['churn'] == 'False.'])
n1 = len(control_group)
m2 = len(treatment2[treatment2['churn'] == 'False.'])
n2 = len(treatment2)

In [27]:
T01 = (m0 / n0 - m1 / n1) / np.sqrt(((m0 + m1) / (n0 + n1)) * (1 - (m0 + m1) / (n0 + n1))
                                    * (1 / n0 + 1 / n1))
T21 = (m2 / n2 - m1 / n1) / np.sqrt(((m1 + m2) / (n1 + n2)) * (1 - (m1 + m2) / (n1 + n2))
                                    * (1 / n1 + 1 / n2))
print(T01)
print(T21)

1.20466885512
2.59905482098


Проведем анализ эффективности удержания. Сначала найдем критическое значение $u$, соответствующее уровню значимости $\alpha = 0.05$. Если мы проверяем равенство эффектов, то $\Phi(u) = 0.5 - \frac{\alpha}{2}$, где $\Phi(x) = \frac{1}{\sqrt{2\pi}}\int\limits_0^x e^{-\frac{t^2}{2}}dt$ - функция Лапласа. У нас $\alpha = 0.05$, и тогда $u = 1.96$.

Следовательно, для $T_{01}$ нулевая гипотеза принимается, так как $|T| < u$, а для $T_{21}$ - отвергается. Это значит, что имеющиеся данные позволяют считать, что метод treatment = 0 не является эффективным, а метод treatment = 2 эффективным является, и его можно использовать.

То же самое можно посчитать с помощью критерия хи-квадрат. Сначала получим таблицу сопряженности для treatment и churn:

In [28]:
observed = []
for treatment in sorted(data['treatment'].unique()):
    observed_treatment = []
    for churn in data['churn'].unique():
        observed_freq = len(data[(data['treatment'] == treatment) &
                                 (data['churn'] == churn)])
        observed_treatment.append(observed_freq)
    observed.append(observed_treatment)
print(observed)

[[968, 165], [917, 180], [965, 138]]


In [29]:
chi2, p, dof, expected = chi2_contingency([observed[0], observed[1]])
chi2

1.3135522978703889

In [30]:
chi2, p, dof, expected = chi2_contingency([observed[2], observed[1]])
chi2

6.4435991416199307

В данном случае статистика аппроксимируется хи-квадрат распределением с одной степенью свободы. Табличное критическое значение при уровне значимости $\alpha = 0.05$ равно $3.84$, поэтому нулевая гипотеза в первом случае принимается, а во втором отвергается, и метод treatment = 0 не эффективен, а treatment = 2 эффективен.